<center> <h1 style="background-color:seagreen; color:white" >Agrupamentos com Kmeans com Spark usando PySpark</h1></center>  

Dados sem classe (não supervisionado)

Agrupamento de forma orgânica (de acordo com algoritmo e seus parâmetros)

<center> <h2 style="background-color:DarkKhaki; color:white" >Inicializando o PySpark</h2></center> 

In [4]:
import findspark
import pyspark
from pyspark.sql import SparkSession

# Faz a Interafce entre o Spark e o Jupyter Notebook
findspark.init()

# Inicializando uma Sessão no Spark
spark = SparkSession.builder.appName("Kmeans").getOrCreate()

<center> <h3 style="background-color:#ECF7AE; color:#3F403D" >Hiper Parâmetros</h3></center> 
   
> + **stanceMeasure**: métrica de distância. Opções Euclidean e cosine.
(padrão: euclidean)
> + **k**: Números de cluster  a serem criados.(padrão: 2)
> + **maxIter**: número maxim de iteração. (padrão: 20)
> + **predictionCol**: nome da coluna com o cluster. (padrão: prediction)



---

<center> <h1 style="background-color:seagreen; color:white" >Aplicando Agrupamentos com Kmeans no DataSet Íris</h1></center>



<center> <h3 style="background-color:#ECF7AE; color:#3F403D" >Carregando o Cunjunto de Dados Íris</h3></center> 

In [19]:
iris = spark.read.csv("../Material_do_Curso/iris.csv",
                             header=True, inferSchema=True, sep=",")
print(f"Quantidade de Registros do Dataset: {iris.count()}")
iris.show(5, truncate=True)

Quantidade de Registros do Dataset: 150
+-----------+----------+-----------+----------+-----------+
|sepallength|sepalwidth|petallength|petalwidth|      class|
+-----------+----------+-----------+----------+-----------+
|        5.1|       3.5|        1.4|       0.2|Iris-setosa|
|        4.9|       3.0|        1.4|       0.2|Iris-setosa|
|        4.7|       3.2|        1.3|       0.2|Iris-setosa|
|        4.6|       3.1|        1.5|       0.2|Iris-setosa|
|        5.0|       3.6|        1.4|       0.2|Iris-setosa|
+-----------+----------+-----------+----------+-----------+
only showing top 5 rows



<center> <h3 style="background-color:#ECF7AE; color:#3F403D" >Escolha das Variáveis Independentes e Dependente para treinamento do Modelo</h3></center> 


<center> <h3 style="background-color:#ECF7AE; color:#3F403D" >Importação do Módulo do PySpark Para o Pré-Processamento dos Dados</h3></center> 

In [20]:
from pyspark.ml.feature import VectorAssembler

In [21]:
# Colocando os dados no Formato de dataframe do Spark usando o VectorAssembler
asb = VectorAssembler(inputCols=["sepallength", "sepalwidth",
                                      "petallength", "petalwidth"],
                      outputCol="features")
iris_rf = asb.transform(iris)
iris_rf.show(5, truncate=False)

+-----------+----------+-----------+----------+-----------+-----------------+
|sepallength|sepalwidth|petallength|petalwidth|class      |features         |
+-----------+----------+-----------+----------+-----------+-----------------+
|5.1        |3.5       |1.4        |0.2       |Iris-setosa|[5.1,3.5,1.4,0.2]|
|4.9        |3.0       |1.4        |0.2       |Iris-setosa|[4.9,3.0,1.4,0.2]|
|4.7        |3.2       |1.3        |0.2       |Iris-setosa|[4.7,3.2,1.3,0.2]|
|4.6        |3.1       |1.5        |0.2       |Iris-setosa|[4.6,3.1,1.5,0.2]|
|5.0        |3.6       |1.4        |0.2       |Iris-setosa|[5.0,3.6,1.4,0.2]|
+-----------+----------+-----------+----------+-----------+-----------------+
only showing top 5 rows



In [22]:
from pyspark.ml.feature import StringIndexer

In [24]:
idx = StringIndexer(inputCol="class", outputCol="grupos")
iris_rfsi = idx.fit(iris_rf).transform(iris_rf)
iris_rfsi.show(5, truncate=False)

+-----------+----------+-----------+----------+-----------+-----------------+------+
|sepallength|sepalwidth|petallength|petalwidth|class      |features         |grupos|
+-----------+----------+-----------+----------+-----------+-----------------+------+
|5.1        |3.5       |1.4        |0.2       |Iris-setosa|[5.1,3.5,1.4,0.2]|0.0   |
|4.9        |3.0       |1.4        |0.2       |Iris-setosa|[4.9,3.0,1.4,0.2]|0.0   |
|4.7        |3.2       |1.3        |0.2       |Iris-setosa|[4.7,3.2,1.3,0.2]|0.0   |
|4.6        |3.1       |1.5        |0.2       |Iris-setosa|[4.6,3.1,1.5,0.2]|0.0   |
|5.0        |3.6       |1.4        |0.2       |Iris-setosa|[5.0,3.6,1.4,0.2]|0.0   |
+-----------+----------+-----------+----------+-----------+-----------------+------+
only showing top 5 rows



<center> <h3 style="background-color:#ECF7AE; color:#3F403D" >Convertendo os valores float da coluna grupos para inteiro</h3></center> 

In [25]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType

iris_rfsi = iris_rfsi.withColumn("grupos", iris_rfsi["grupos"].cast(IntegerType()))
iris_rfsi.show(5)

+-----------+----------+-----------+----------+-----------+-----------------+------+
|sepallength|sepalwidth|petallength|petalwidth|      class|         features|grupos|
+-----------+----------+-----------+----------+-----------+-----------------+------+
|        5.1|       3.5|        1.4|       0.2|Iris-setosa|[5.1,3.5,1.4,0.2]|     0|
|        4.9|       3.0|        1.4|       0.2|Iris-setosa|[4.9,3.0,1.4,0.2]|     0|
|        4.7|       3.2|        1.3|       0.2|Iris-setosa|[4.7,3.2,1.3,0.2]|     0|
|        4.6|       3.1|        1.5|       0.2|Iris-setosa|[4.6,3.1,1.5,0.2]|     0|
|        5.0|       3.6|        1.4|       0.2|Iris-setosa|[5.0,3.6,1.4,0.2]|     0|
+-----------+----------+-----------+----------+-----------+-----------------+------+
only showing top 5 rows



---


<center> <h3 style="background-color:#ECF7AE; color:#3F403D" > Importação do Módulo do PySpark Para Criação do Modelo de Agrupamentos com Kmeans</h3></center> 


In [26]:
from pyspark.ml.clustering import KMeans

#### Instanciando Objeto e criando o Modelo

In [27]:
# Instanciando o objeto KMeans
obj_km = KMeans(featuresCol="features", predictionCol="pre_grupo", maxIter=100, k=3)

# Criando o Modelo
model_km = obj_km.fit(iris_rfsi)

In [29]:
grupos = model_km.transform(iris_rfsi)

In [30]:
grupos.show(150)

+-----------+----------+-----------+----------+---------------+-----------------+------+---------+
|sepallength|sepalwidth|petallength|petalwidth|          class|         features|grupos|pre_grupo|
+-----------+----------+-----------+----------+---------------+-----------------+------+---------+
|        5.1|       3.5|        1.4|       0.2|    Iris-setosa|[5.1,3.5,1.4,0.2]|     0|        1|
|        4.9|       3.0|        1.4|       0.2|    Iris-setosa|[4.9,3.0,1.4,0.2]|     0|        1|
|        4.7|       3.2|        1.3|       0.2|    Iris-setosa|[4.7,3.2,1.3,0.2]|     0|        1|
|        4.6|       3.1|        1.5|       0.2|    Iris-setosa|[4.6,3.1,1.5,0.2]|     0|        1|
|        5.0|       3.6|        1.4|       0.2|    Iris-setosa|[5.0,3.6,1.4,0.2]|     0|        1|
|        5.4|       3.9|        1.7|       0.4|    Iris-setosa|[5.4,3.9,1.7,0.4]|     0|        1|
|        4.6|       3.4|        1.4|       0.3|    Iris-setosa|[4.6,3.4,1.4,0.3]|     0|        1|
|        5

<center> <h1 style="background-color:seagreen; color:white" >Avaliando o Modelo</h1></center> 
<br>

In [32]:
classe = grupos.select("grupos").collect()
agrupado = grupos.select("pre_grupo").collect()

from sklearn.metrics import confusion_matrix, accuracy_score

cm = confusion_matrix(classe, agrupado)
print(cm)

[[ 0 50  0]
 [ 2  0 48]
 [36  0 14]]


In [34]:
acuracia = (cm[2,0] + cm[0, 1] + cm[1, 2])/150
print(f"Acurácia: {acuracia}")

Acurácia: 0.8933333333333333


---